Build and train a CNN+MLP deep learning model with Keras with two followings for MNIST dataset:
```
1. Conv2D(32, kernel_size=(3, 3), activation='relu')
2. Conv2D(64, kernel_size=(3, 3), activation='relu')
3. MaxPooling2D(pool_size=(2, 2))
4. Dense(128, activation='relu')
5. Dense(num_classes, activation='softmax')
```
Also build another model with BatchNormalization and Dropout. 
Compare these two models performance for test data

## Import Packages

In [88]:
import keras
# CNN and MLP architecture
from keras.models import Sequential
from keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    UpSampling2D,
    Dropout,
    Flatten
)
from keras.models import Model
from keras import backend as K
import numpy as np
from keras.optimizers import SGD
from keras.initializers import RandomNormal
# MNIST
from keras.datasets import mnist
# Data normalization
from sklearn.preprocessing import StandardScaler

## 1 - Data Preparation

In [94]:
# Split Data into Train and Test
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Flattening Image
# x_train = np.reshape(x_train, [-1, 28*28])  # we want 784 samples
# x_test = np.reshape(x_test,[-1, 28*28])

#reshape data to fit model
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

# Displaying Resulting Dimensions
print(f'Shape of X_train: {x_train.shape}')
print(f'Shape of X_test: {x_test.shape}')

Shape of X_train: (60000, 28, 28, 1)
Shape of X_test: (10000, 28, 28, 1)


## 2 - Data Normalization

In [95]:
x_train = x_train/np.max(x_train)
x_test = x_test/np.max(x_train)

## 3 - One Hot Encoding

In [96]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [97]:
# store the number of labels 
num_classes = len(np.unique(y_train))

## 4 - Define Model (No Batch Normalization/Dropout)

In [98]:
# Instanitate a model using the Sequential API
fully_connected = Sequential()

# Convolutional Layers
fully_connected.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                           input_shape=(28, 28, 1)))
fully_connected.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
fully_connected.add(MaxPooling2D(pool_size=(2, 2)))  # no learning params
fully_connected.add(Flatten())

# MLP Layers
fully_connected.add(Dense(128, activation='relu'))
fully_connected.add(Dense(num_classes, activation='softmax'))

# Compile Model
fully_connected.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train the model
fully_connected.fit(x_train, y_train,
                    epochs=20, batch_size=100,
                    validation_data=(x_test, y_test),
                    verbose=0)

# Print Summary
fully_connected.summary()

ValueError: Error when checking target: expected dense_58 to have shape (2,) but got array with shape (10,)